# Attention Mechanism

> Fill in a module description here

In [ ]:
#| default_exp transformer.attention

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
import torch
from torch import nn

In [ ]:
def addition(a,b):
    "Adds two numbers together"
    return a+b

#| tip a+b

We take the sum of a and b, which is written in python with the "+" symbol

In [ ]:
#| export
class A:
    pass

In [ ]:
from fastcore.foundation import docs

In [ ]:
@docs
class PrepareForMultiHeadAttention(nn.Module):
    def __init__(self, d_model, heads, d_k, bias):
        super().__init__()
        self.linear = nn.Linear(d_model, heads * d_k, bias=bias)
        self.heads = heads
        self.d_k = d_k

    def forward(self, x):
        head_shape = x.shape[:-1]
        
        x = self.linear(x)
        x = x.view(*head_shape, self.heads, self.d_k)
        
        return x
    
    _docs = dict(cls_doc="xxx",
                 forward="yyy")

#| explain "self.heads = heads"

dasdasd

### Multi-head Attention

In practice, we don't compute each attention score at once, but we concentrate all the `key` to one matrix, same for `value` and `query`. Then calculate all the attention score of these at once.

$$\operatorname{Attention}(Q, K, V)=\underset{\text { seq }}{\operatorname{softmax}}\left(\frac{Q K^{\top}}{\sqrt{d_k}}\right) V$$

- `d_model`: the number of features in `query`, `key`, and `value` vectors.
- `heads`: the number of attention layers.
- `d_k`: the number of dimension of each vector in each head

In [ ]:
#| exports
class MultiHeadAttention(nn.Module):
    def __init__(
        self,
        heads: int,
        d_model: int,
        dropout_prop: float=0.1,
        bias: bool = True
    ):
        super().__init__()
        self.d_k = d_model // heads
        self.heads = heads
        self.query = PrepareForMultiHeadAttention(d_model, heads, self.d_k, bias)